In [1]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv('./머신러닝 전처리 데이터.CSV',encoding='EUC-KR')

In [19]:
data

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,31,2023-05-31,5,23,16,3.5,2920010200,1,22350
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,9,2023-05-09,5,22,16,3.5,2920010200,1,22400
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,13,2023-05-13,13,22,16,3.5,2920010200,1,22000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425074,광주광역시 서구 화정동,화정우미,115.1850,200611,10,2006-11-10,17,10,131,4.5,2914011900,1,19000
425075,광주광역시 서구 화정동,화정우미,84.8300,200611,27,2006-11-27,6,10,131,4.5,2914011900,1,12000
425076,광주광역시 서구 화정동,화정우미,59.8300,200611,30,2006-11-30,2,10,131,4.5,2914011900,1,7700
425077,광주광역시 서구 화정동,화정우미,84.8400,200612,6,2006-12-06,16,10,124,4.5,2914011900,1,12170


In [5]:
pip install xgboost

                                              0.0/70.9 MB ? eta -:--:--
                                              0.3/70.9 MB 7.9 MB/s eta 0:00:09
                                              1.0/70.9 MB 10.9 MB/s eta 0:00:07
                                              1.8/70.9 MB 12.3 MB/s eta 0:00:06
     -                                        2.6/70.9 MB 15.3 MB/s eta 0:00:05
     -                                        3.1/70.9 MB 13.3 MB/s eta 0:00:06
     --                                       4.1/70.9 MB 15.4 MB/s eta 0:00:05
     --                                       5.1/70.9 MB 15.4 MB/s eta 0:00:05
     ---                                      5.4/70.9 MB 14.5 MB/s eta 0:00:05
     ---                                      6.2/70.9 MB 14.8 MB/s eta 0:00:05
     ----                                     7.4/70.9 MB 15.8 MB/s eta 0:00:05
     ----                                     8.1/70.9 MB 15.6 MB/s eta 0:00:05
     -----                                    9.1

In [6]:
pip install lightgbm

                                              0.0/1.3 MB ? eta -:--:--
     -------------                            0.5/1.3 MB 14.2 MB/s eta 0:00:01
     -----------------------------            1.0/1.3 MB 12.3 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 10.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tqdm

                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import re

In [13]:
data.head(2)

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.31,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.98,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000


In [5]:
data[data['시군구']=='광주광역시 남구 봉선동']

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
2104,광주광역시 남구 봉선동,금호2,122.8100,202301,12,2023-01-12,15,28,29,3.5,2915511500,0,64000
2105,광주광역시 남구 봉선동,금호2,122.8100,202301,15,2023-01-15,1,28,29,3.5,2915511500,0,60500
2106,광주광역시 남구 봉선동,금호2,84.9600,202301,21,2023-01-21,6,28,29,3.5,2915511500,0,49500
2107,광주광역시 남구 봉선동,금호2,84.8300,202303,25,2023-03-25,6,28,27,3.5,2915511500,0,49500
2108,광주광역시 남구 봉선동,금호2,84.8300,202304,18,2023-04-18,13,28,27,3.5,2915511500,0,49000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
406940,광주광역시 남구 봉선동,포스코더샵,59.8900,200612,23,2006-12-23,5,2,100,4.5,2915511500,0,13650
406941,광주광역시 남구 봉선동,포스코더샵,59.8900,200612,27,2006-12-27,17,2,100,4.5,2915511500,0,13000
406942,광주광역시 남구 봉선동,포스코더샵,84.3200,200612,27,2006-12-27,8,2,100,4.5,2915511500,0,23900
406943,광주광역시 남구 봉선동,포스코더샵,150.9700,200612,27,2006-12-27,2,2,100,4.5,2915511500,0,41000


In [6]:
data['시군구']

0         광주광역시 광산구 도산동
1         광주광역시 광산구 도산동
2         광주광역시 광산구 도산동
3         광주광역시 광산구 도산동
4         광주광역시 광산구 도산동
              ...      
425074     광주광역시 서구 화정동
425075     광주광역시 서구 화정동
425076     광주광역시 서구 화정동
425077     광주광역시 서구 화정동
425078     광주광역시 서구 화정동
Name: 시군구, Length: 425079, dtype: object

In [9]:
X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]
y = data['거래금액(만원)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# dacon 하이퍼파라미터튜닝 모델. 연습용
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=100, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1 )
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=9, min_child_weight=5, n_jobs=-1,num_leaves=20)

In [10]:
# 특성없음
rf = RandomForestRegressor()

In [11]:
def print_rmse_r2_score(model):
    # cv별 프린팅, 평균 저장
    model_name, rmse_list, r2_list = rmse_r2_cv(model)
    for i, (rmse, r2) in enumerate(zip(rmse_list, r2_list), start=1):
         print(f'{i} FOLDS: {model_name} RMSE: {rmse:.4f}, R2: {r2:.4f}')
    print(f'\n{model_name} mean RMSE: {np.mean(rmse_list):.4f}, mean R2: {np.mean(r2_list):.4f}')
    print('='*40)
    return model_name, np.mean(rmse_list), np.mean(r2_list)

def rmse_r2_cv(model):
    # cv별로 학습하는 함수
    tscv = TimeSeriesSplit(n_splits=10)
    rmse_list = []
    r2_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(X), start=1), desc=f'{model_name} Cross Validations...', total=10):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        rmse = RMSE(y_test, pred)
        r2 = r2_score(y_test, pred)
        rmse_list.append(rmse)
        r2_list.append(r2)
    return model_name, rmse_list, r2_list

def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

In [12]:
models = []
rmse_scores = []
r2_scores = []
for model in [rf]:
    model_name, mean_rmse_score, mean_r2_score = print_rmse_r2_score(model)
    models.append(model_name)
    rmse_scores.append(mean_rmse_score)
    r2_scores.append(mean_r2_score)

RandomForestRegressor Cross Validations...: 100%|█████████████████████████████████████| 10/10 [20:20<00:00, 122.06s/it]

1 FOLDS: RandomForestRegressor RMSE: 7803.8514, R2: 0.7901
2 FOLDS: RandomForestRegressor RMSE: 5418.1207, R2: 0.8322
3 FOLDS: RandomForestRegressor RMSE: 4845.6045, R2: 0.8172
4 FOLDS: RandomForestRegressor RMSE: 3709.6537, R2: 0.8415
5 FOLDS: RandomForestRegressor RMSE: 3034.8998, R2: 0.8532
6 FOLDS: RandomForestRegressor RMSE: 2576.1283, R2: 0.8965
7 FOLDS: RandomForestRegressor RMSE: 3635.5145, R2: 0.8198
8 FOLDS: RandomForestRegressor RMSE: 3322.2844, R2: 0.8103
9 FOLDS: RandomForestRegressor RMSE: 1949.1586, R2: 0.8748
10 FOLDS: RandomForestRegressor RMSE: 1754.1990, R2: 0.8655

RandomForestRegressor mean RMSE: 3804.9415, mean R2: 0.8401


In [8]:
result_df = pd.DataFrame({'Model': models, 'RMSE': rmse_scores, 'R2': r2_scores}).reset_index(drop=True)
result_df
# 하이퍼파라미터 튜닝 없는 자료

,Model,RMSE,R2
0,LinearRegression,6722.620306,0.385872
1,Ridge,6722.618718,0.385872
2,Lasso,6722.256191,0.385947
3,ElasticNet,6669.493452,0.394279
4,DecisionTreeRegressor,5032.619546,0.726100
5,RandomForestRegressor,3807.453119,0.840822
6,XGBRegressor,3451.422637,0.864900
7,LGBMRegressor,3583.182899,0.852412


In [12]:
# 기존dacon 하이퍼파라미터 튜닝 자료 
result_df = pd.DataFrame({'Model': models, 'RMSE': rmse_scores, 'R2': r2_scores}).reset_index(drop=True)
result_df

,Model,RMSE,R2
0,LinearRegression,6722.620306,0.385872
1,Ridge,6722.619036,0.385872
2,Lasso,6722.616620,0.385873
3,ElasticNet,6715.504320,0.387007
4,DecisionTreeRegressor,5866.145420,0.602225
5,RandomForestRegressor,4549.020550,0.760308
6,XGBRegressor,3298.991401,0.880218
7,LGBMRegressor,3700.854867,0.841873


In [13]:
X_predict = [[84, 202305,10,10 ,15,3.5,2920010200,0]]
X_predict_df= pd.DataFrame(X_predict,columns=['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수'])

X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]

In [15]:
print(f"LinearRegression: {lrmodel.predict(X_predict_df)[0]}")
print(f"Ridge: {ridge.predict(X_predict_df)[0]}")
print(f"Lasso: {lasso.predict(X_predict_df)[0]}")
print(f"ElasticNet: {enet.predict(X_predict_df)[0]}")
print(f"DecisionTreeRegressor: {d_tree.predict(X_predict_df)[0]}")
print(f"RandomForestRegressor: {rf.predict(X_predict_df)[0]}")
print(f"XGBRegressor: {model_xgb.predict(X_predict_df)[0]}")
print(f"LGBMRegressor :{model_lgb.predict(X_predict_df)[0]}")

LinearRegression: 36369.40774595132
Ridge: 36369.40581294848
Lasso: 36369.36288404139
ElasticNet: 36342.717726878356
DecisionTreeRegressor: 37111.30126002291
RandomForestRegressor: 38480.428627831374
XGBRegressor: 33039.31640625
LGBMRegressor :37965.11034514065


In [22]:
lrmodel = LinearRegression()
ridge = Ridge()
lasso = Lasso()
enet = ElasticNet()
d_tree = DecisionTreeRegressor()
rf = RandomForestRegressor()
model_xgb = xgb.XGBRegressor()
model_lgb = lgb.LGBMRegressor()

In [ ]:
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=100, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1 )
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=9, min_child_weight=5, n_jobs=-1,num_leaves=20)

##  하이퍼 파라미터 튜닝

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
# Linear Regression
lr_params = {
    'fit_intercept': [True, False]
}
lr_grid_search = GridSearchCV(estimator=LinearRegression(n_jobs=-1),
                              param_grid=lr_params,
                              scoring='neg_mean_squared_error',
                              cv=10,
                              n_jobs=-1)
lr_grid_search.fit(X, y)
best_lr_model = lr_grid_search.best_estimator_

KeyboardInterrupt: 

In [12]:


# Ridge
ridge_params = {
    'alpha': [0.1, 0.5, 1.0, 2.0]
}
ridge_grid_search = GridSearchCV(estimator=Ridge(random_state=1),
                                 param_grid=ridge_params,
                                 scoring='neg_mean_squared_error',
                                 cv=10,
                                 n_jobs=-1)
ridge_grid_search.fit(X, y)
best_ridge_model = ridge_grid_search.best_estimator_

In [13]:
# Lasso
lasso_params = {
    'alpha': [0.001, 0.01, 0.1, 1.0]
}
lasso_grid_search = GridSearchCV(estimator=Lasso(random_state=1),
                                 param_grid=lasso_params,
                                 scoring='neg_mean_squared_error',
                                 cv=10,
                                 n_jobs=-1)
lasso_grid_search.fit(X, y)
best_lasso_model = lasso_grid_search.best_estimator_

In [14]:
# ElasticNet
enet_params = {
    'alpha': [0.001, 0.01, 0.1, 1.0],
    'l1_ratio': [0.01, 0.1, 0.5, 0.9]
}
enet_grid_search = GridSearchCV(estimator=ElasticNet(random_state=1),
                                param_grid=enet_params,
                                scoring='neg_mean_squared_error',
                                cv=10,
                                n_jobs=-1)
enet_grid_search.fit(X, y)
best_enet_model = enet_grid_search.best_estimator_

In [15]:
# Decision Tree
d_tree_params = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}
d_tree_grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=1),
                                  param_grid=d_tree_params,
                                  scoring='neg_mean_squared_error',
                                  cv=10,
                                  n_jobs=-1)
d_tree_grid_search.fit(X, y)
best_d_tree_model = d_tree_grid_search.best_estimator_

In [15]:
# RandomForest
rf_params = {
    'n_estimators': [50, 70, 100],
    'max_depth': [5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}
rf_grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=1, n_jobs=-1),
                              param_grid=rf_params,
                              scoring='neg_mean_squared_error',
                              cv=10,
                              n_jobs=-1)
rf_grid_search.fit(X, y)
best_rf_model = rf_grid_search.best_estimator_


In [ ]:
# XGBoost
xgb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_child_weight': [1, 3, 5],
    'gamma': [0.1, 0.2, 0.3]
}
xgb_grid_search = GridSearchCV(estimator=xgb.XGBRegressor(n_jobs=-1),
                               param_grid=xgb_params,
                               scoring='neg_mean_squared_error',
                               cv=10,
                               n_jobs=-1)
xgb_grid_search.fit(X, y)
best_xgb_model = xgb_grid_search.best_estimator_


In [ ]:

# LightGBM
lgb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_child_weight': [1, 3, 5],
    'num_leaves': [10, 20, 30]
}
lgb_grid_search = GridSearchCV(estimator=lgb.LGBMRegressor(n_jobs=-1),
                               param_grid=lgb_params,
                               scoring='neg_mean_squared_error',
                               cv=10,
                               n_jobs=-1)
lgb_grid_search.fit(X, y)
best_lgb_model = lgb_grid_search.best_estimator_



In [17]:
# 출력



print("Best RandomForest Model:", best_rf_model)
# print("Best XGBoost Model:", best_xgb_model)
# print("Best LightGBM Model:", best_lgb_model)

Best RandomForest Model: RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=50,
                      n_jobs=-1, random_state=1)


In [18]:
print("Best RandomForest Model:", best_rf_model)

Best RandomForest Model: RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=50,
                      n_jobs=-1, random_state=1)
